In [1]:
using InteractiveUtils, Pkg
using AdvancedHMC; Pkg.activate(pkgdir(AdvancedHMC) * "/research/tests"); Pkg.instantiate()
versioninfo(); Pkg.status()

  Activating project at `~/.julia/dev/AdvancedHMC/research/tests`


Julia Version 1.9.0-rc2
Commit 72aec423c2a (2023-04-01 10:41 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin21.4.0)
  CPU: 20 × Apple M1 Ultra
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, westmere)
  Threads: 2 on 20 virtual cores
Status `~/.julia/dev/AdvancedHMC/research/tests/Project.toml`
  [c75e803d] AdaptiveRejectionSampling v0.1.1
  [0bf59076] AdvancedHMC v0.4.5
  [6e4b80f9] BenchmarkTools v1.3.2
  [863f3e99] Comonicon v1.0.4
  [163ba53b] DiffResults v1.1.0
  [31c24e10] Distributions v0.25.87
  [366bfd00] DynamicPPL v0.22.2
  [6a86dc24] FiniteDiff v2.19.0
  [f6369f11] ForwardDiff v0.10.35
  [6d524b87] MCMCDebugging v0.2.1 `https://github.com/TuringLang/MCMCDebugging.jl#master`
  [91a5bcdd] Plots v1.38.9
  [d330b81b] PyPlot v2.11.1
  [e0db7c4e] ReTest v0.3.2
  [37e2e3b7] ReverseDiff v1.14.4
  [a8a75453] StatProfilerHTML v1.5.0
  [8a639fad] VecTargets v0.2.0 `https://github.com/xukai92/VecTargets.jl#main`


In [2]:
using LinearAlgebra, FiniteDiff, VecTargets

In [3]:
target = Funnel()

ℓπ = x -> logpdf(target, x)
neg_ℓπ = x -> -logpdf(target, x)

H = x -> VecTargets.gen_hess(ℓπ, x)(x)[3]
G = x -> VecTargets.gen_hess(neg_ℓπ, x)(x)[3]

#9 (generic function with 1 method)

In [4]:
xt = [0.0, 0.0] # x test

@show H(xt) G(xt)

# Hamiltonian_partial(x) = begin x
#     lad, s = logabsdet(G(x))
#     lad * s
# end # WRONG implementation of the second term of (13)
      # `s` returned is the sign of `det(G)`, not the whole thing
Hamiltonian_partial(x) = begin x
    logdet(G(x))
end # second term of (13)

@show FiniteDiff.finite_difference_gradient(Hamiltonian_partial, xt)

Jt = FiniteDiff.finite_difference_jacobian(G, xt)
@show [tr(inv(G(xt)) * Jt[1:2,1:2]), tr(inv(G(xt)) * Jt[3:4,1:2])]

H(xt) = [-0.1111111111111111 0.0; 0.0 -1.0]
G(xt) = [0.1111111111111111 0.0; 0.0 1.0]
FiniteDiff.finite_difference_gradient(Hamiltonian_partial, xt) = [-0.9999999999502168, 0.0]
[tr(inv(G(xt)) * Jt[1:2, 1:2]), tr(inv(G(xt)) * Jt[3:4, 1:2])] = [-0.9999999664723873, 0.0]


2-element Vector{Float64}:
 -0.9999999664723873
  0.0